<a href="https://colab.research.google.com/github/Provatiscus/Coursera_Capstone/blob/master/Battle_of_the_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook will be mainly used for the capstone project.


# Fourth Part

In [79]:
CLIENT_ID = '42RLQPYKMQ22V5TUBQVRENTR5NLNNHLLVGFPVPNXHX2JEITE' # your Foursquare ID
CLIENT_SECRET = 'GYZMZKBOTMXQGD5DRVRYFGRI33FRUL2JM1AKKTKART53WWIS' # your Foursquare Secret
ACCESS_TOKEN = 'R4CZYCVKYSPFY0ITDXXEA0WAPC2IJIWYJCZOXI1HATUFO1OY' # your FourSquare Access
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')
!pip install geopy

from geopy.geocoders import Nominatim
nom = Nominatim(user_agent="Peutiblond")
nom.geocode('Chartres, Centre-Ville')

Your credentails:
CLIENT_ID: 42RLQPYKMQ22V5TUBQVRENTR5NLNNHLLVGFPVPNXHX2JEITE
CLIENT_SECRET:GYZMZKBOTMXQGD5DRVRYFGRI33FRUL2JM1AKKTKART53WWIS
/bin/bash: conda: command not found
Libraries imported.


Location(Chemin du Puits de la Ville, Épernon, Chartres, Eure-et-Loir, Centre-Val de Loire, France métropolitaine, 28230, France, (48.6163534, 1.67576, 0.0))

In [80]:
def get_venues2(latitude, longitude):
  url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
  results = requests.get(url).json()
  results
  items = results['response']['groups'][0]['items']
  return items
radius = 500

In [81]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

chartres_page = 'https://immobilier-chartres.nestenn.com/quartiers-chartres'
page = urlopen(chartres_page)
soup = BeautifulSoup(page, 'html.parser')


In [82]:
zero_data = np.zeros((29,5))
dfC = pd.DataFrame(zero_data)

In [83]:
dfC.columns = ["Quartier", "nbMaison", "nbHabitants", "nbLocataires", "nbProprietaires"]
for col in dfC.columns:
  dfC[col] = range(29)

In [84]:
def fn_quartier(i):
  return soup.find_all(class_="titreQuartier")[i].text
def fn_maison(i):
  return soup.find_all(class_="nbMaison")[i].text
def fn_hab(i):
  return soup.find_all(class_="nbHabitants")[i].text
def fn_loc(i):
  return soup.find_all(class_="nbLocataires")[i].text
def fn_pro(i):
  return soup.find_all(class_="nbProprietaires")[i].text


In [85]:
dfC["Quartier"] = dfC["Quartier"].apply(fn_quartier)
dfC["nbMaison"] = dfC["nbMaison"].apply(fn_maison)
dfC["nbHabitants"] = dfC["nbHabitants"].apply(fn_hab)
dfC["nbLocataires"] = dfC["nbLocataires"].apply(fn_loc)
dfC["nbProprietaires"] = dfC["nbProprietaires"].apply(fn_pro)

In [86]:
dfC.head()

,Quartier,nbMaison,nbHabitants,nbLocataires,nbProprietaires
0,Centre-Ville,3677,6231,2250,1359
1,Hotel Dieu,2107,4062,1246,1039
2,Grand faubourg,2107,4062,1246,1039
3,Hotel de Ville,1933,2968,1235,574
4,Saint-pierre,1933,2968,1235,574


In [89]:
def flat(quartier):
  if nom.geocode('Chartres ' + quartier) != None:
    #print(nom.geocode('Chartres ' + quartier))
    return nom.geocode('Chartres ' + quartier).latitude
  else:
    return None

In [90]:
def flong(quartier):
  if nom.geocode('Chartres ' + quartier) != None:
    #print(nom.geocode('Chartres ' + quartier))
    return nom.geocode('Chartres ' + quartier).longitude
  else:
    return None

In [91]:
dfC['latitude'] = dfC['Quartier'].apply(flat)

In [92]:
dfC['longitude'] = dfC['Quartier'].apply(flong)

In [95]:
dfC

,Quartier,nbMaison,nbHabitants,nbLocataires,nbProprietaires,latitude,longitude
0,Centre-Ville,3677,6231,2250,1359,NaN,NaN
1,Hotel Dieu,2107,4062,1246,1039,48.440960,1.480725
2,Grand faubourg,2107,4062,1246,1039,48.443634,1.482033
3,Hotel de Ville,1933,2968,1235,574,48.443801,1.489084
4,Saint-pierre,1933,2968,1235,574,48.448406,1.480907
5,Cathedrale,1426,2131,737,511,48.447812,1.487910
6,Saint-Andre,1426,2131,737,511,48.450128,1.489943
7,Coteau d'Aboville,924,3279,958,550,48.450870,1.497473
8,Comtesses,1114,2478,766,630,48.435139,1.485373
9,Villaines,1114,2478,766,630,48.436264,1.481752


In [96]:
centre = nom.geocode('Chartres Centre ')
dfC.loc[0, "latitude"] = centre.latitude
dfC.loc[0, "longitude"] = centre.longitude

In [97]:
dfC.drop(11, inplace = True)

In [99]:
dfC = dfC.reset_index(drop = True)
dfC.head()

,Quartier,nbMaison,nbHabitants,nbLocataires,nbProprietaires,latitude,longitude
0,Centre-Ville,3677,6231,2250,1359,48.042346,-1.706558
1,Hotel Dieu,2107,4062,1246,1039,48.440960,1.480725
2,Grand faubourg,2107,4062,1246,1039,48.443634,1.482033
3,Hotel de Ville,1933,2968,1235,574,48.443801,1.489084
4,Saint-pierre,1933,2968,1235,574,48.448406,1.480907


In [100]:
latitude = dfC.loc[1,'latitude']
longitude = dfC.loc[1,'longitude']
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco

for lat, lng, label in zip(dfC['latitude'], dfC['longitude'], dfC['Quartier']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)


venues_map

## Looks like Geopy got it wrong on entries 11,23,25 and 12, we drop those entries

In [101]:
dfC.drop([11, 23,25, 12], inplace = True)

In [102]:
dfC = dfC.reset_index(drop = True)

In [103]:
dfC.head()

,Quartier,nbMaison,nbHabitants,nbLocataires,nbProprietaires,latitude,longitude
0,Centre-Ville,3677,6231,2250,1359,48.042346,-1.706558
1,Hotel Dieu,2107,4062,1246,1039,48.440960,1.480725
2,Grand faubourg,2107,4062,1246,1039,48.443634,1.482033
3,Hotel de Ville,1933,2968,1235,574,48.443801,1.489084
4,Saint-pierre,1933,2968,1235,574,48.448406,1.480907


In [104]:
latitude = dfC.loc[1,'latitude']
longitude = dfC.loc[1,'longitude']
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco

for lat, lng, label in zip(dfC['latitude'], dfC['longitude'], dfC['Quartier']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)


venues_map

In [105]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)



In [29]:
dfC_venues = getNearbyVenues(names=dfC['Quartier'],
                                   latitudes=dfC['latitude'],
                                   longitudes=dfC['longitude']
                                  )

Centre-Ville
Hotel Dieu
Grand faubourg
Hotel de Ville
Saint-pierre
Cathedrale
Saint-Andre
Coteau d'Aboville
Comtesses
Villaines
Madeleine
Chanzy
Petite Venise
Saint-Brice
Saint-Cheron
Croix Bonnard
Bas Bourgs
Lavoisier
Petits Clos
Beaulieu 
Rechevres
Proust
Clos Brette
Bas Menus


In [110]:
dfC_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Centre-Ville,48.042346,-1.706558,Ozan Kebab,48.041819,-1.706296,Kebab Restaurant
1,Centre-Ville,48.042346,-1.706558,optique de la marionnais,48.041942,-1.705520,Optical Shop
2,Centre-Ville,48.042346,-1.706558,Les Blés d'Or,48.041984,-1.702462,French Restaurant
3,Hotel Dieu,48.440960,1.480725,Best Western Le Grand Monarque Hotel Chartres,48.444390,1.482929,Hotel
4,Hotel Dieu,48.440960,1.480725,Théâtre de Chartres,48.442809,1.485891,Theater


In [111]:
len(dfC_venues["Venue Category"].unique())

56

In [112]:
dfC_venues[dfC_venues['Venue Category']=='Hotel']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
3,Hotel Dieu,48.440960,1.480725,Best Western Le Grand Monarque Hotel Chartres,48.444390,1.482929,Hotel
12,Grand faubourg,48.443634,1.482033,Best Western Le Grand Monarque Hotel Chartres,48.444390,1.482929,Hotel
38,Hotel de Ville,48.443801,1.489084,Best Western Le Grand Monarque Hotel Chartres,48.444390,1.482929,Hotel
56,Saint-pierre,48.448406,1.480907,Best Western Le Grand Monarque Hotel Chartres,48.444390,1.482929,Hotel
103,Saint-Andre,48.450128,1.489943,Ibis Chartres Centre Cathédrale,48.452876,1.489478,Hotel


In [113]:
dfC_onehot = pd.get_dummies(dfC_venues[['Venue Category']], prefix="", prefix_sep="")
dfC_grouped = dfC_venues[["Neighborhood"]].join(dfC_onehot).groupby("Neighborhood").sum()
pd.DataFrame(dfC_grouped.sum(axis = 1),columns = ['Count']).head()

,Count
Neighborhood,
Bas Bourgs,5
Bas Menus,1
Beaulieu,5
Cathedrale,28
Centre-Ville,3


In [114]:
dfC_grouped.head()

,BBQ Joint,Bakery,Bar,Basketball Stadium,Bookstore,Brasserie,Breakfast Spot,Business Service,Café,Campground,Clothing Store,Cocktail Bar,Convenience Store,Creperie,Department Store,Fast Food Restaurant,Flower Shop,French Restaurant,Furniture / Home Store,Garden,Gas Station,Gastropub,Hardware Store,Historic Site,Home Service,Hostel,Hotel,Hotel Bar,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Lawyer,Miscellaneous Shop,Mobile Phone Shop,Optical Shop,Park,Perfume Shop,Pharmacy,Pizza Place,Plaza,Rental Car Location,Restaurant,Sandwich Place,Sculpture Garden,Shopping Mall,Skate Park,Soccer Field,Soccer Stadium,Squash Court,Stadium,Steakhouse,Supermarket,Tea Room,Tennis Stadium,Theater,Train Station
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Bas Bourgs,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Bas Menus,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Beaulieu,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
Cathedrale,0,1,0,0,0,0,1,0,1,0,0,0,0,1,1,1,0,4,0,1,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,1,1,0,1,4,0,2,1,0,0,0,0,0,0,0,1,0,1,0,0,1
Centre-Ville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [115]:
dfC_onehot.head()

,BBQ Joint,Bakery,Bar,Basketball Stadium,Bookstore,Brasserie,Breakfast Spot,Business Service,Café,Campground,Clothing Store,Cocktail Bar,Convenience Store,Creperie,Department Store,Fast Food Restaurant,Flower Shop,French Restaurant,Furniture / Home Store,Garden,Gas Station,Gastropub,Hardware Store,Historic Site,Home Service,Hostel,Hotel,Hotel Bar,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Lawyer,Miscellaneous Shop,Mobile Phone Shop,Optical Shop,Park,Perfume Shop,Pharmacy,Pizza Place,Plaza,Rental Car Location,Restaurant,Sandwich Place,Sculpture Garden,Shopping Mall,Skate Park,Soccer Field,Soccer Stadium,Squash Court,Stadium,Steakhouse,Supermarket,Tea Room,Tennis Stadium,Theater,Train Station
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [116]:
import json

with open("drive/MyDrive/Capstone/Chartres.geojson", 'r') as C:
  C_geo = C.read()

C_geojson = json.loads(C_geo)

In [117]:
with open("drive/MyDrive/Capstone/Chartres.geojson", 'r') as C:
  C_geo = C.read()
latitude = dfC.loc[1,'latitude']
longitude = dfC.loc[1,'longitude']
C_geo
C_map = folium.Map(location=[latitude, longitude], zoom_start=12)
C_map.choropleth(
    geo_data=C_geojson,
    #data=df_incident_grouped,
    #columns=['Neighborhood', 'Count'],
    #key_on='feature.properties.DISTRICT',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Crime Rate in San Francisco',

)
for lat, lng, label in zip(dfC['latitude'], dfC['longitude'], dfC['Quartier']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(C_map)




C_map

/usr/local/lib/python3.6/dist-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [118]:
np.array(C_geojson['features'][0]['geometry']['coordinates']).mean(axis = 1)

array([[ 1.48738821, 48.44831144]])

In [120]:
dfC_grouped2=pd.DataFrame(np.zeros((len(C_geojson['features']),dfC_grouped.shape[1])))
dfC_grouped2.columns = dfC_grouped.columns

In [121]:
index = []
for e in C_geojson['features']:
  index = index + [e['properties']['nom_iris']]
dfC_grouped2.index = index

In [122]:
dfC_grouped2.loc["Cathedrale Saint-André"] = dfC_grouped.loc["Cathedrale"] + dfC_grouped.loc["Saint-Andre"]
dfC_grouped2.loc["Bas Menus Rechèvres"] = dfC_grouped.loc["Bas Menus"] + dfC_grouped.loc["Rechevres"]
dfC_grouped2.loc["Hotel de Ville Saint-pierre"] = dfC_grouped.loc["Hotel de Ville"] + dfC_grouped.loc["Saint-pierre"]
dfC_grouped2.loc["Beaulieu Clos Brette"] = dfC_grouped.loc["Beaulieu "] + dfC_grouped.loc["Clos Brette"]
dfC_grouped2.loc["Saint-Chéron"] = dfC_grouped.loc["Saint-Cheron"]
dfC_grouped2.loc["Saint-jean Rechèvres Bourgneuf"] = dfC_grouped.loc["Rechevres"]
dfC_grouped2.loc["Coteau d'Aboville"] = dfC_grouped.loc["Coteau d'Aboville"]
dfC_grouped2.loc["Madeleine Briand"] = dfC_grouped.loc["Madeleine"]
dfC_grouped2.loc["Madeleine Proust"] = dfC_grouped.loc["Madeleine"] + dfC_grouped.loc["Proust"]
dfC_grouped2.loc["Croix Bonnard Bas Bourgs"] = dfC_grouped.loc["Croix Bonnard"] + dfC_grouped.loc["Bas Bourgs"]
dfC_grouped2.loc["Hotel Dieu Grand faubourg"] = dfC_grouped.loc["Hotel Dieu"] + dfC_grouped.loc["Grand faubourg"]
dfC_grouped2.loc["Comtesses Villaines"] = dfC_grouped.loc["Comtesses"] + dfC_grouped.loc["Villaines"]
dfC_grouped2.loc["Chanzy Petite Venise"] = dfC_grouped.loc["Chanzy"] + dfC_grouped.loc["Petite Venise"]
dfC_grouped2.loc["Beaulieu Lavoisier Petits Clos"] = dfC_grouped.loc["Beaulieu "] + dfC_grouped.loc["Lavoisier"] + dfC_grouped.loc["Petits Clos"]


In [123]:
dfC_grouped2.head()

,BBQ Joint,Bakery,Bar,Basketball Stadium,Bookstore,Brasserie,Breakfast Spot,Business Service,Café,Campground,Clothing Store,Cocktail Bar,Convenience Store,Creperie,Department Store,Fast Food Restaurant,Flower Shop,French Restaurant,Furniture / Home Store,Garden,Gas Station,Gastropub,Hardware Store,Historic Site,Home Service,Hostel,Hotel,Hotel Bar,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Lawyer,Miscellaneous Shop,Mobile Phone Shop,Optical Shop,Park,Perfume Shop,Pharmacy,Pizza Place,Plaza,Rental Car Location,Restaurant,Sandwich Place,Sculpture Garden,Shopping Mall,Skate Park,Soccer Field,Soccer Stadium,Squash Court,Stadium,Steakhouse,Supermarket,Tea Room,Tennis Stadium,Theater,Train Station
Cathedrale Saint-André,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,8.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,6.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
Bas Menus Rechèvres,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Hotel de Ville Saint-pierre,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,1.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,4.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
Beaulieu Clos Brette,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
Saint-Chéron,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [124]:
dfC_sum = dfC_grouped2.sum(axis = 1)
dfC_sum = pd.DataFrame(dfC_sum)
dfC_sum["Quartier"] = dfC_sum.index
dfC_sum.columns = ["Count", "Quartier"]
dfC_sum

,Count,Quartier
Cathedrale Saint-André,40.0,Cathedrale Saint-André
Bas Menus Rechèvres,5.0,Bas Menus Rechèvres
Hotel de Ville Saint-pierre,37.0,Hotel de Ville Saint-pierre
Beaulieu Clos Brette,8.0,Beaulieu Clos Brette
Saint-Chéron,4.0,Saint-Chéron
Saint-jean Rechèvres Bourgneuf,4.0,Saint-jean Rechèvres Bourgneuf
Coteau d'Aboville,4.0,Coteau d'Aboville
Madeleine Briand,9.0,Madeleine Briand
Hauts de Chartres,0.0,Hauts de Chartres
Madeleine Proust,10.0,Madeleine Proust


In [125]:
with open("drive/MyDrive/Capstone/Chartres.geojson", 'r') as C:
  C_geo = C.read()
latitude = dfC.loc[1,'latitude']
longitude = dfC.loc[1,'longitude']
C_geo
C_map = folium.Map(location=[latitude, longitude], zoom_start=12)
C_map.choropleth(
    geo_data=C_geojson,
    data=dfC_sum,
    columns=['Quartier', 'Count'],
    key_on='feature.properties.nom_iris',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='number of venues in Chartres',

)
# for lat, lng, label in zip(dfC['latitude'], dfC['longitude'], dfC['Quartier']):
#     folium.CircleMarker(
#         [lat, lng],
#         radius=5,
#         popup=label,
#         fill=True,
#         color='blue',
#         fill_color='blue',
#         fill_opacity=0.6
#         ).add_to(C_map)

C_map


/usr/local/lib/python3.6/dist-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


# PRICE


In [126]:
df_price = pd.DataFrame(index = dfC_grouped2.index, columns = ['Price'])

In [127]:
df_price['Price'] = [2212, 2097, 2212, 1848, 1923, 2097, 2039, 1843, 1834, 1843, 2224, 2173, 2224, 1923, 1848, 1843]

In [128]:
df_price['Quartier'] = df_price.index
df_price

,Price,Quartier
Cathedrale Saint-André,2212,Cathedrale Saint-André
Bas Menus Rechèvres,2097,Bas Menus Rechèvres
Hotel de Ville Saint-pierre,2212,Hotel de Ville Saint-pierre
Beaulieu Clos Brette,1848,Beaulieu Clos Brette
Saint-Chéron,1923,Saint-Chéron
Saint-jean Rechèvres Bourgneuf,2097,Saint-jean Rechèvres Bourgneuf
Coteau d'Aboville,2039,Coteau d'Aboville
Madeleine Briand,1843,Madeleine Briand
Hauts de Chartres,1834,Hauts de Chartres
Madeleine Proust,1843,Madeleine Proust


In [129]:
df_price['Price'] = [2212, 2097, 2212, 1848, 1923, 2097, 2039, 1843, 1834, 1843, 2224, 2173, 2224, 2220, 1848, 1843]
with open("drive/MyDrive/Capstone/Chartres.geojson", 'r') as C:
  C_geo = C.read()
latitude = dfC.loc[1,'latitude']
longitude = dfC.loc[1,'longitude']
C_geo
C_map = folium.Map(location=[latitude, longitude], zoom_start=12)
C_map.choropleth(
    geo_data=C_geojson,
    data=df_price,
    columns=['Quartier', 'Price'],
    key_on='feature.properties.nom_iris',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Average price in in the Neighborhood',

)

C_map

/usr/local/lib/python3.6/dist-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


#Clustering

In [130]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dfC_grouped.index

for ind in np.arange(dfC_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dfC_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted



,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bas Bourgs,Rental Car Location,Bar,Business Service,Park,Train Station,Home Service,Historic Site,Hardware Store,Gastropub,Gas Station
1,Bas Menus,Pharmacy,Train Station,Hotel Bar,Hostel,Home Service,Historic Site,Hardware Store,Gastropub,Gas Station,Garden
2,Beaulieu,Tennis Stadium,Historic Site,Squash Court,Campground,Train Station,Department Store,Home Service,Hardware Store,Gastropub,Gas Station
3,Cathedrale,Plaza,French Restaurant,Restaurant,Train Station,Breakfast Spot,Café,Creperie,Department Store,Fast Food Restaurant,Garden
4,Centre-Ville,Kebab Restaurant,Optical Shop,French Restaurant,Train Station,Hotel,Home Service,Historic Site,Hardware Store,Gastropub,Gas Station
5,Chanzy,Theater,Home Service,Hardware Store,Business Service,Fast Food Restaurant,Hotel,Hostel,Historic Site,Gastropub,Gas Station
6,Clos Brette,Supermarket,Historic Site,Cocktail Bar,Train Station,Department Store,Hostel,Home Service,Hardware Store,Gastropub,Gas Station
7,Comtesses,Business Service,Train Station,Department Store,Hostel,Home Service,Historic Site,Hardware Store,Gastropub,Gas Station,Garden
8,Coteau d'Aboville,Hostel,Stadium,Soccer Field,Sculpture Garden,Creperie,Home Service,Historic Site,Hardware Store,Gastropub,Gas Station
9,Croix Bonnard,Campground,Train Station,Department Store,Hostel,Home Service,Historic Site,Hardware Store,Gastropub,Gas Station,Garden


In [131]:
# set number of clusters
kclusters = 5

#dfC_grouped2_clustering = dfC_grouped2.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dfC_grouped2)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 0, 1, 3, 0, 0, 0, 4, 0, 4, 0, 2, 0, 0, 3, 0], dtype=int32)

In [132]:
dfC_grouped2

,BBQ Joint,Bakery,Bar,Basketball Stadium,Bookstore,Brasserie,Breakfast Spot,Business Service,Café,Campground,Clothing Store,Cocktail Bar,Convenience Store,Creperie,Department Store,Fast Food Restaurant,Flower Shop,French Restaurant,Furniture / Home Store,Garden,Gas Station,Gastropub,Hardware Store,Historic Site,Home Service,Hostel,Hotel,Hotel Bar,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Lawyer,Miscellaneous Shop,Mobile Phone Shop,Optical Shop,Park,Perfume Shop,Pharmacy,Pizza Place,Plaza,Rental Car Location,Restaurant,Sandwich Place,Sculpture Garden,Shopping Mall,Skate Park,Soccer Field,Soccer Stadium,Squash Court,Stadium,Steakhouse,Supermarket,Tea Room,Tennis Stadium,Theater,Train Station
Cathedrale Saint-André,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,8.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,6.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
Bas Menus Rechèvres,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Hotel de Ville Saint-pierre,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,1.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,4.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
Beaulieu Clos Brette,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
Saint-Chéron,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Saint-jean Rechèvres Bourgneuf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Coteau d'Aboville,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
Madeleine Briand,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
Hauts de Chartres,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Madeleine Proust,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0


In [133]:
dfC_grouped2["Cluster"] = kmeans.labels_

In [134]:
dfC_grouped2["Cluster"]==0

Cathedrale Saint-André            False
Bas Menus Rechèvres                True
Hotel de Ville Saint-pierre       False
Beaulieu Clos Brette              False
Saint-Chéron                       True
Saint-jean Rechèvres Bourgneuf     True
Coteau d'Aboville                  True
Madeleine Briand                  False
Hauts de Chartres                  True
Madeleine Proust                  False
Croix Bonnard Bas Bourgs           True
Hotel Dieu Grand faubourg         False
Comtesses Villaines                True
Chanzy Petite Venise               True
Beaulieu Lavoisier Petits Clos    False
Madeleine Sablons                  True
Name: Cluster, dtype: bool

In [135]:
# add clustering labels
dfC_cluster = dfC_grouped2[["BBQ Joint"]]
#dfC_cluster.insert(0, 'Cluster Labels', kmeans.labels_)
dfC_cluster["Neighborhoods"]= dfC_cluster.index
dfC_cluster["Cluster"]=kmeans.labels_
dfC_cluster.drop(["BBQ Joint"], axis = 1, inplace = True)
dfC_cluster

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

,Neighborhoods,Cluster
Cathedrale Saint-André,Cathedrale Saint-André,1
Bas Menus Rechèvres,Bas Menus Rechèvres,0
Hotel de Ville Saint-pierre,Hotel de Ville Saint-pierre,1
Beaulieu Clos Brette,Beaulieu Clos Brette,3
Saint-Chéron,Saint-Chéron,0
Saint-jean Rechèvres Bourgneuf,Saint-jean Rechèvres Bourgneuf,0
Coteau d'Aboville,Coteau d'Aboville,0
Madeleine Briand,Madeleine Briand,4
Hauts de Chartres,Hauts de Chartres,0
Madeleine Proust,Madeleine Proust,4


In [139]:
def add_lat_long(nei, row):
  dfC_cluster.loc[nei, 'latitude'] = dfC.loc[row, 'latitude']
  dfC_cluster.loc[nei, 'longitude'] = dfC.loc[row, 'longitude']
  #print(nei, row)

In [140]:
add_lat_long("Cathedrale Saint-André", 5)
add_lat_long("Bas Menus Rechèvres", 20)
add_lat_long("Hotel de Ville Saint-pierre", 3)
add_lat_long("Beaulieu Clos Brette", 19)
add_lat_long("Saint-Chéron", 14)
add_lat_long("Saint-jean Rechèvres Bourgneuf", 18)
add_lat_long("Coteau d'Aboville", 6)
add_lat_long("Madeleine Briand", 7)
add_lat_long("Hauts de Chartres", 8)
add_lat_long("Madeleine Proust", 9)
add_lat_long("Croix Bonnard Bas Bourgs", 10)
add_lat_long("Hotel Dieu Grand faubourg", 11)
add_lat_long("Comtesses Villaines", 12)
add_lat_long("Chanzy Petite Venise", 13)
add_lat_long("Beaulieu Lavoisier Petits Clos", 17)
add_lat_long("Madeleine Sablons", 21)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [142]:
kclusters =5
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dfC_cluster['latitude'], dfC_cluster['longitude'], dfC_cluster['Neighborhoods'], dfC_cluster['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [143]:
dfC_grouped2 = dfC_venues[["Neighborhood", "Venue Latitude", "Venue Longitude"]]
dfC_grouped2.head()

,Neighborhood,Venue Latitude,Venue Longitude
0,Centre-Ville,48.041819,-1.706296
1,Centre-Ville,48.041942,-1.705520
2,Centre-Ville,48.041984,-1.702462
3,Hotel Dieu,48.444390,1.482929
4,Hotel Dieu,48.442809,1.485891


In [145]:
dfC_grouped.head()

,BBQ Joint,Bakery,Bar,Basketball Stadium,Bookstore,Brasserie,Breakfast Spot,Business Service,Café,Campground,Clothing Store,Cocktail Bar,Convenience Store,Creperie,Department Store,Fast Food Restaurant,Flower Shop,French Restaurant,Furniture / Home Store,Garden,Gas Station,Gastropub,Hardware Store,Historic Site,Home Service,Hostel,Hotel,Hotel Bar,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Lawyer,Miscellaneous Shop,Mobile Phone Shop,Optical Shop,Park,Perfume Shop,Pharmacy,Pizza Place,Plaza,Rental Car Location,Restaurant,Sandwich Place,Sculpture Garden,Shopping Mall,Skate Park,Soccer Field,Soccer Stadium,Squash Court,Stadium,Steakhouse,Supermarket,Tea Room,Tennis Stadium,Theater,Train Station
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Bas Bourgs,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Bas Menus,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Beaulieu,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
Cathedrale,0,1,0,0,0,0,1,0,1,0,0,0,0,1,1,1,0,4,0,1,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,1,1,0,1,4,0,2,1,0,0,0,0,0,0,0,1,0,1,0,0,1
Centre-Ville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [146]:
with open("drive/MyDrive/Capstone/Chartres.geojson", 'r') as C:
  C_geo = C.read()
latitude = dfC.loc[1,'latitude']
longitude = dfC.loc[1,'longitude']
C_geo
C_map = folium.Map(location=[latitude, longitude], zoom_start=12)
C_map.choropleth(
    geo_data=C_geojson,
    data=dfC_sum,
    columns=['Quartier', 'Count'],
    key_on='feature.properties.nom_iris',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='number of venues in Chartres',)


x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dfC_cluster['latitude'], dfC_cluster['longitude'], dfC_cluster['Neighborhoods'], dfC_cluster['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(C_map)
       
C_map

/usr/local/lib/python3.6/dist-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [77]:
https://nbviewer.jupyter.org/github/Provatiscus/Coursera_Capstone/blob/master/Battle_of_the_Neighborhoods.ipynb